# Preparation of external data for figures

In [9]:
import pathlib
from datetime import datetime
import numpy as np
import xarray as xr
import satpy
from satpy import Scene


In [3]:
data_path = pathlib.Path("../data")
if not data_path.exists():
    data_path.mkdir()

## SEVIRI image data

In [4]:
seviri_ds = xr.Dataset()

In [7]:
seviri_path = pathlib.Path("/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/l1")
seviri_files = sorted(list(seviri_path.glob("MSG3-SEVI-MSG15-0100-NA-20160601*.nat")))
print(seviri_files)
scn = Scene(reader="seviri_l1b_native", filenames=[str(f) for f in seviri_files])
scn.load(['VIS006', 'VIS008', 'IR_016', "WV_062", "WV_073", "IR_108", "IR_120"])
b = scn.get("VIS006").data.reshape([4,-1,3712])[:,::-1,::-1]
g = scn.get("VIS008").data.reshape([4,-1,3712])[:,::-1,::-1]
r = scn.get("IR_016").data.reshape([4,-1,3712])[:,::-1,::-1]
rgb = np.minimum(np.maximum(np.stack([r, g, b], -1)/100, 0), 1)
bt = scn.get("IR_108").data.reshape([4,-1,3712])[:,::-1,::-1]
wvd = scn.get("WV_062").data.reshape([4,-1,3712])[:,::-1,::-1] - scn.get("WV_073").data.reshape([4,-1,3712])[:,::-1,::-1]
swd = scn.get("IR_108").data.reshape([4,-1,3712])[:,::-1,::-1] - scn.get("IR_120").data.reshape([4,-1,3712])[:,::-1,::-1]

[PosixPath('/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/l1/MSG3-SEVI-MSG15-0100-NA-20160601121241.710000000Z-NA.nat'), PosixPath('/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/l1/MSG3-SEVI-MSG15-0100-NA-20160601151241.890000000Z-NA.nat'), PosixPath('/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/l1/MSG3-SEVI-MSG15-0100-NA-20160601181242.132000000Z-NA.nat'), PosixPath('/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/l1/MSG3-SEVI-MSG15-0100-NA-20160601211241.225000000Z-NA.nat')]


/home/users/wkjones/miniconda3/envs/tobac_flow/lib/python3.11/site-packages/satpy/readers/seviri_base.py:489: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  dataset.coords['acq_time'] = ('y', acq_time)
/home/users/wkjones/miniconda3/envs/tobac_flow/lib/python3.11/site-packages/satpy/readers/seviri_base.py:489: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-

In [8]:
rgb

dask.array<minimum, shape=(4, 3712, 3712, 3), dtype=float32, chunksize=(1, 3712, 3712, 1), chunktype=numpy.ndarray>

In [10]:
coords = {
    "time":[datetime(2016,6,1,12), datetime(2016,6,1,15), datetime(2016,6,1,18), datetime(2016,6,1,21)]
}
dims = ("time", "y", "x")

In [11]:
seviri_ds["rgb"] = xr.DataArray(rgb, coords=coords, dims=dims+("channel",))
seviri_ds["bt"] = xr.DataArray(bt, dims=dims)
seviri_ds["wvd"] = xr.DataArray(wvd, dims=dims)
seviri_ds["swd"] = xr.DataArray(swd, dims=dims)

In [12]:
seviri_ds

<xarray.Dataset>
Dimensions:  (time: 4, y: 3712, x: 3712, channel: 3)
Coordinates:
  * time     (time) datetime64[ns] 2016-06-01T12:00:00 ... 2016-06-01T21:00:00
Dimensions without coordinates: y, x, channel
Data variables:
    rgb      (time, y, x, channel) float32 dask.array<chunksize=(1, 3712, 3712, 1), meta=np.ndarray>
    bt       (time, y, x) float32 dask.array<chunksize=(1, 3712, 3712), meta=np.ndarray>
    wvd      (time, y, x) float32 dask.array<chunksize=(1, 3712, 3712), meta=np.ndarray>
    swd      (time, y, x) float32 dask.array<chunksize=(1, 3712, 3712), meta=np.ndarray>

In [13]:
seviri_ds.to_netcdf(data_path / "seviri_image_data.nc")

## Cloud CCI flux data example

In [14]:
flx_path = pathlib.Path("/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/flx")
flx_file = list(flx_path.glob("2016060115*.nc"))[0]
flx_ds = xr.open_dataset(flx_file)
from tobac_flow.postprocess import add_cre_to_dataset
flx_ds = add_cre_to_dataset(flx_ds)

/home/users/wkjones/miniconda3/envs/tobac_flow/lib/python3.11/site-packages/xarray/coding/times.py:237: CFWarning: this date/calendar/year zero convention is not supported by CF
  cftime.num2date(num_dates, units, calendar, only_use_cftime_datetimes=True)


In [15]:
flx_ds

<xarray.Dataset>
Dimensions:       (along_track: 1601, across_track: 2081)
Dimensions without coordinates: along_track, across_track
Data variables: (12/37)
    time          (along_track, across_track) datetime64[ns] ...
    lat           (along_track, across_track) float32 ...
    lon           (along_track, across_track) float32 ...
    retrflag      (along_track, across_track) float32 ...
    toa_swdn      (along_track, across_track) float32 ...
    toa_swup      (along_track, across_track) float32 ...
    ...            ...
    boa_lwdn_cre  (along_track, across_track) float32 114.0 80.68 ... 0.0 0.0
    boa_lwup_cre  (along_track, across_track) float32 0.0 0.0 0.0 ... 0.0 0.0
    toa_net       (along_track, across_track) float32 -239.7 -240.7 ... 814.8
    toa_net_cre   (along_track, across_track) float32 34.33 32.79 ... -0.0 -0.0
    boa_net       (along_track, across_track) float32 -1.798 -34.95 ... 841.4
    boa_net_cre   (along_track, across_track) float32 114.0 80.68 ... 0.0 0.0
Attributes: (12/36)
    Conventions:            CF-1.6,ACDD-1.3
    title:                  ESA Cloud CCI Retrieval Products L2 Primary File
    institution:            Deutscher Wetterdienst
    source:                 null
    history:                Dataset produced by DWDs CC4CL retrieval system i...
    references:             https://climate.esa.int/en/projects/cloud/
    ...                     ...
    Emissivity_File:        /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...
    USGS_File:              /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...
    Snow_File:              /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...
    Sea_Ice_File:           /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...
    Level1b_File:           /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...
    Geo_File:               /scratch/ms/de/sf7/esa_cci_c_proc/CCFLOW_SEV_DEV/...

In [16]:
flx_ds.to_netcdf(data_path / "broadband_flux_data.nc")

/home/users/wkjones/miniconda3/envs/tobac_flow/lib/python3.11/site-packages/xarray/coding/times.py:237: CFWarning: this date/calendar/year zero convention is not supported by CF
  cftime.num2date(num_dates, units, calendar, only_use_cftime_datetimes=True)


## Calibration flux data

In [18]:
# CERES Fluxes
from dateutil.parser import parse as parse_date
ceres_path = pathlib.Path("/gws/nopw/j04/eo_shared_data_vol1/satellite/ceres/Data/EBAF")
ceres_files = sorted(list(ceres_path.glob("CERES_EBAF_Ed4.2_Subset_20160[5678]*.nc")))
ceres_ds = xr.open_mfdataset(ceres_files, combine="nested", concat_dim="time")
ceres_ds = ceres_ds.roll(lon=180).assign_coords(lon=((ceres_ds.lon.roll(lon=180) + 180) % 360) -180)
ceres_ds = ceres_ds.groupby("time.month").mean("time")

flx_path = pathlib.Path("/gws/nopw/j04/eo_shared_data_vol2/satellite/seviri-orac/flx_grid/")
flx_files = sorted(list(flx_path.glob("*.nc")))
gridded_flx_ds = xr.open_mfdataset(flx_files, combine="nested", concat_dim="t")
gridded_flx_ds = gridded_flx_ds.assign_coords(t=[parse_date(str(f)[-18:-10]) for f in flx_files])
gridded_flx_ds = gridded_flx_ds.sel(t = gridded_flx_ds.t[gridded_flx_ds.n_times>92])
print(f'Valid days: {gridded_flx_ds.t.size}')
gridded_flx_ds = gridded_flx_ds.groupby("t.month").mean("t")


Valid days: 106


In [19]:
ceres_ds

<xarray.Dataset>
Dimensions:                      (lat: 180, month: 4, lon: 360)
Coordinates:
  * lat                          (lat) float32 -89.5 -88.5 -87.5 ... 88.5 89.5
  * lon                          (lon) float32 -179.5 -178.5 ... 178.5 179.5
  * month                        (month) int64 5 6 7 8
Data variables: (12/41)
    toa_sw_all_mon               (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_lw_all_mon               (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_net_all_mon              (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_sw_clr_c_mon             (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_lw_clr_c_mon             (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_net_clr_c_mon            (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ...                           ...
    sfc_net_tot_all_mon          (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sfc_net_tot_clr_c_mon        (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sfc_net_tot_clr_t_mon        (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sfc_cre_net_sw_mon           (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sfc_cre_net_lw_mon           (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    sfc_cre_net_tot_mon          (month, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes:
    title:        CERES EBAF TOA and Surface Fluxes. Monthly Averages and 07/...
    institution:  NASA Langley Research Center
    Conventions:  CF-1.4
    comment:      Climatology from 07/2005 to 06/2015
    version:      Edition 4.2; Revised data Release Date January 2, 2024
    DOI:          10.5067/TERRA-AQUA-NOAA20/CERES/EBAF_L3B004.2

In [20]:
gridded_flx_ds

<xarray.Dataset>
Dimensions:       (lat: 180, lon: 360, month: 4)
Coordinates:
  * lat           (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon           (lon) float64 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * month         (month) int64 5 6 7 8
Data variables: (12/23)
    n_times       (month) float64 95.84 95.69 95.71 95.65
    toa_swup      (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_swup_clr  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_swup_cre  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_lwup      (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    toa_lwup_clr  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ...            ...
    boa_lwdn_cre  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    boa_lwup      (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    boa_lwup_clr  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    boa_lwup_cre  (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    boa_net       (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    boa_net_cre   (month, lat, lon) float64 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>

In [22]:
img_extent = (-19, 47, -33, 16)
lons = slice(img_extent[0], img_extent[1])
lats = slice(img_extent[2], img_extent[3])
ceres_ds.toa_cre_net_mon.sel(lon=lons, lat=lats)

<xarray.DataArray 'toa_cre_net_mon' (month: 4, lat: 49, lon: 66)>
dask.array<getitem, shape=(4, 49, 66), dtype=float32, chunksize=(1, 49, 66), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -32.5 -31.5 -30.5 -29.5 -28.5 ... 12.5 13.5 14.5 15.5
  * lon      (lon) float32 -18.5 -17.5 -16.5 -15.5 -14.5 ... 43.5 44.5 45.5 46.5
  * month    (month) int64 5 6 7 8
Attributes:
    long_name:      Top of The Atmosphere Cloud Radiative Effects Net Flux, (...
    standard_name:  TOA CRE Net Flux
    CF_name:        toa_net_cloud_radiative_effect
    comment:        Calculated as all-sky flux minus clear-sky (for total reg...
    units:          W m-2
    valid_min:           -400.000
    valid_max:            400.000

In [23]:
gridded_flx_ds.toa_net_cre.sel(lon=lons, lat=lats)

<xarray.DataArray 'toa_net_cre' (month: 4, lat: 49, lon: 66)>
dask.array<getitem, shape=(4, 49, 66), dtype=float64, chunksize=(1, 49, 66), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -32.5 -31.5 -30.5 -29.5 -28.5 ... 12.5 13.5 14.5 15.5
  * lon      (lon) float64 -18.5 -17.5 -16.5 -15.5 -14.5 ... 43.5 44.5 45.5 46.5
  * month    (month) int64 5 6 7 8

In [24]:
calibration_flx_ds = xr.Dataset()

img_extent = (-19, 47, -33, 16)
lons = slice(img_extent[0], img_extent[1])
lats = slice(img_extent[2], img_extent[3])

calibration_flx_ds["ceres_toa_net_cre"] = ceres_ds.toa_cre_net_mon.sel(lon=lons, lat=lats)
calibration_flx_ds["ceres_toa_sw_cre"] = ceres_ds.toa_cre_sw_mon.sel(lon=lons, lat=lats)
calibration_flx_ds["ceres_toa_lw_cre"] = ceres_ds.toa_cre_lw_mon.sel(lon=lons, lat=lats)

calibration_flx_ds["seviri_toa_net_cre"] = gridded_flx_ds.toa_net_cre.sel(lon=lons, lat=lats)
calibration_flx_ds["seviri_toa_sw_cre"] = gridded_flx_ds.toa_swup_cre.sel(lon=lons, lat=lats)
calibration_flx_ds["seviri_toa_lw_cre"] = gridded_flx_ds.toa_lwup_cre.sel(lon=lons, lat=lats)


In [25]:
calibration_flx_ds

<xarray.Dataset>
Dimensions:             (lat: 49, lon: 66, month: 4)
Coordinates:
  * lat                 (lat) float64 -32.5 -31.5 -30.5 -29.5 ... 13.5 14.5 15.5
  * lon                 (lon) float64 -18.5 -17.5 -16.5 -15.5 ... 44.5 45.5 46.5
  * month               (month) int64 5 6 7 8
Data variables:
    ceres_toa_net_cre   (month, lat, lon) float32 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>
    ceres_toa_sw_cre    (month, lat, lon) float32 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>
    ceres_toa_lw_cre    (month, lat, lon) float32 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>
    seviri_toa_net_cre  (month, lat, lon) float64 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>
    seviri_toa_sw_cre   (month, lat, lon) float64 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>
    seviri_toa_lw_cre   (month, lat, lon) float64 dask.array<chunksize=(1, 49, 66), meta=np.ndarray>

In [26]:
calibration_flx_ds.to_netcdf(data_path / "calibration_data.nc")

## Land/Sea mask

In [27]:
# Get land/sea mask from imerg
lsm_file = "/home/users/wkjones/GPM_IMERG_LandSeaMask.2.nc4"
lsm_ds = xr.open_dataset(lsm_file)

lsm = lsm_ds.landseamask

wh_sea = lsm >=100 

In [29]:
lsm_ds = xr.Dataset()
lsm_ds["sea_mask"] = wh_sea

In [30]:
lsm_ds

<xarray.Dataset>
Dimensions:   (lon: 3600, lat: 1800)
Coordinates:
  * lon       (lon) float64 -179.9 -179.8 -179.7 -179.6 ... 179.8 179.9 180.0
  * lat       (lat) float64 -89.95 -89.85 -89.75 -89.65 ... 89.75 89.85 89.95
Data variables:
    sea_mask  (lon, lat) bool False False False False ... True True True True

In [31]:
lsm_ds.to_netcdf(data_path / "lsm_data.nc")